In [ ]:
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset

In [ ]:
kitti_sequences_path = Path('/datasets/SemanticKITTI/dataset/sequences/')
sequence_paths = sorted(list(kitti_sequences_path.iterdir()))

In [ ]:
sequence_indices = {
    'trainval': set(range(0, 11)),
    'test': set(range(11, 22)),
}

sequence_indices['valid'] = {8}

sequence_indices['train'] = sequence_indices['trainval'] - sequence_indices['valid']
del sequence_indices['trainval']

In [ ]:
splits = ['train', 'valid', 'test']
sequence_id_strs = {
    split: [f'{i:02d}' for i in sequence_indices[split]]
    for split in splits
}

In [ ]:
def lidar_path(sequence_index: int, frame_index: int):
    return kitti_sequences_path / f'{sequence_index:02d}' / 'velodyne' / f'{frame_index:06d}.bin'

def label_path(sequence_index: int, frame_index: int):
    return kitti_sequences_path / f'{sequence_index:02d}' / 'labels' / f'{frame_index:06d}.label'

In [ ]:
def num_frames(sequence_index):
    path = kitti_sequences_path / f'{sequence_index:02d}' / 'velodyne'
    return len(list(path.iterdir()))

In [ ]:
all_lidar_paths = {
    split : {
        index: sorted((kitti_sequences_path / f'{index:02d}' / 'velodyne').iterdir())
        for index in sequence_indices[split]
    }
    for split in splits
}

all_label_paths = {
    split : {
        index: sorted((kitti_sequences_path / f'{index:02d}' / 'labels').iterdir())
        for index in sequence_indices[split]
    }
    for split in splits
    if split != 'test'
}

In [ ]:
lidar_voxel_length = {
    split: {
        index: len(all_lidar_paths[split][index])
        for index in sequence_indices[split]
    }
    for split in splits
}

In [ ]:
label_voxel_length = {
    split: {
        index: len(all_label_paths[split][index])
        for index in sequence_indices[split]
    }
    for split in splits
    if split != 'test'
}

# Read data

## Read lidara

In [ ]:
with open(all_lidar_paths['train'][0][0]) as lidar_bin:
    scan = np.fromfile(lidar_bin, dtype=np.float32).reshape(-1, 4)
coord = scan[:, :3]
strength = scan[:, 3:4]

In [ ]:
scan

In [ ]:
coord.shape

In [ ]:
strength.shape

In [ ]:
np.histogram(strength)

## Read label

In [ ]:
with open(all_label_paths['train'][0][0]) as label_bin:
    segment = np.fromfile(label_bin, dtype=np.int32)

In [ ]:
segment.shape

In [ ]:
np.unique(segment & 0xFFFF)